In [34]:
import pandas as pd
import datetime
import warnings
warnings.filterwarnings("ignore")  # 忽略UserWarning兼容性警告

# 获取日期字符串
sheet_name1 = (datetime.date.today() - datetime.timedelta(days=2)).strftime("%Y-%m-%d")
sheet_name2 = (datetime.date.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
sheet_name3 = datetime.date.today().strftime("%Y-%m-%d")


# 定义折扣
Discount1 = float('0.76')  # 前天折扣
Discount2 = float('0.76')  # 昨天折扣
Discount3 = float('0.76')  # 今天折扣


floder = r'C:/Users/15516/Desktop/数据处理/竞对数据/牙E在线/牙E在线'

df_yae_front = pd.read_csv(f'{floder}{sheet_name1}.csv', encoding='GBK')
df_yae_yesterday = pd.read_csv(f'{floder}{sheet_name2}.csv', encoding='GBK')
df_yae_now = pd.read_csv(f'{floder}{sheet_name3}.csv', encoding='GBK')

In [36]:
def deal_with(df,Discount):
    # 选择指定列：'链接'、'型号'、'商品编号'、'价格'和'库存'
    df_shaixuan = df[['链接', '型号', '商品编号', '价格', '库存',"品牌","一级分类","二级分类","三级分类"]]

    # 将列名'链接'改为'品名'
    df_shaixuan.rename(columns={'链接': '品名'}, inplace=True)

    # 计算折扣价格，使用变量'Discount1'乘以'价格'列的值
    df_shaixuan['折扣价格'] = Discount * df_shaixuan['价格']

    # 将'商品编号'列的数据类型转换为字符串
    df_shaixuan['商品编号'] = df_shaixuan['商品编号'].astype(str)

    # 创建新列'name'，将'品名'和'型号'列的值相加
    df_shaixuan['name'] = df_shaixuan['品名'] + df_shaixuan['型号']

    # 创建新列'品型'，将'品名'、'型号'和'商品编号'列的值相加
    df_shaixuan['品型'] = df_shaixuan['品名'] + df_shaixuan['型号'] + df_shaixuan['商品编号']

    # 选择指定列：'name'、'品型'、'折扣价格'和'库存'
    df_shaixuan = df_shaixuan[[ '品型', '折扣价格', '库存',"品牌","一级分类","二级分类","三级分类"]]
    
    return df_shaixuan

def Deal_with_file(x, y):
    """
    处理两个DataFrame的合并和计算
    :param x: 第一个DataFrame
    :param y: 第二个DataFrame
    :return: 合并并计算后的DataFrame
    """
    # 合并两个DataFrame
    merged_df = pd.concat([x, y])

    # 去除重复的品型行，保留唯一值
    deduplicated_df = merged_df[['品型']].drop_duplicates()

    # 根据品型列进行左连接
    deduplicated_df = deduplicated_df.merge(x, how='left', on='品型').merge(y, how='left', on='品型')

    # 计算库存变化列和估算金额列
    deduplicated_df['库存变化'] = deduplicated_df.apply(lambda row: calculate_stock_change(row['库存_x'], row['库存_y']),axis=1)
    
    
    deduplicated_df['估算金额'] = deduplicated_df['库存变化'] * deduplicated_df['折扣价格']

    return deduplicated_df

In [39]:
df_yae_front = deal_with(df_yae_front,Discount1)
df_yae_yesterday = deal_with(df_yae_yesterday,Discount2)
df_yae_now = deal_with(df_yae_now,Discount3)

In [41]:
df_last = Deal_with_file(df_yae_front,df_yae_yesterday)


NameError: name 'calculate_stock_change' is not defined

In [45]:
merged_df = pd.concat([df_yae_front, df_yae_yesterday])

# 去除重复的品型行，保留唯一值
deduplicated_df = merged_df[['品型']].drop_duplicates()

# 根据品型列进行左连接
deduplicated_df = deduplicated_df.merge(df_yae_front, how='left', on='品型').merge(df_yae_yesterday, how='left', on='品型')



In [46]:
deduplicated_df

,品型,折扣价格_x,库存_x,品牌_x,一级分类_x,二级分类_x,三级分类_x,折扣价格_y,库存_y,品牌_y,一级分类_y,二级分类_y,三级分类_y
0,雅克菱 天然蛋白酶美白牙粉50g抹茶味 （粉色）3260009002,13.68,42.0,安徽雅克菱,预防/护理,口腔清洁用品,牙粉,13.68,42.0,安徽雅克菱,预防/护理,口腔清洁用品,牙粉
1,雅克菱 天然蛋白酶美白牙粉50g薄荷味（绿色）3260009002,13.68,45.0,安徽雅克菱,预防/护理,口腔清洁用品,牙粉,13.68,45.0,安徽雅克菱,预防/护理,口腔清洁用品,牙粉
2,严迪 皓齿便携式冲牙器喷头补充装（新包装）3*1FC1593270052001,11.78,140.0,广州严迪,预防/护理,口腔清洁用品,冲牙喷嘴,11.78,140.0,广州严迪,预防/护理,口腔清洁用品,冲牙喷嘴
3,GUM Y型牙线棒/签30*130枚200016001,28.88,1037.0,日本GUM,预防/护理,口腔清洁用品,牙线签,28.88,1037.0,日本GUM,预防/护理,口腔清洁用品,牙线签
4,诊所定制 护理牙线棒（5.4元/盒 1000盒起订）50*12000盒/箱780042002,12160.00,500.0,四川效力爽,预防/护理,口腔清洁用品,牙线签,12160.00,500.0,四川效力爽,预防/护理,口腔清洁用品,牙线签
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13904,中国牙谷华柚 EDTA根管润滑剂5ml5ml2930274001,NaN,NaN,NaN,NaN,NaN,NaN,32.68,212.0,四川华柚,牙髓/根管,根管清洗润滑消毒,根管润滑材
13905,淀山湖/青普 聚羧酸锌水门汀30g粉+15ml液4680008001,NaN,NaN,NaN,NaN,NaN,NaN,14.82,244.0,上海青普,牙髓/根管,盖髓垫底洞衬材料,聚羧酸锌类
13906,埃蒙迪 正畸金属托槽/MBT直丝弓金属托槽5-5（含丝）345带钩 KIT11-322-00...,NaN,NaN,NaN,NaN,NaN,NaN,75.24,988.0,上海埃蒙迪,矫正正畸,托槽,金属普通托槽
13907,埃蒙迪 正畸金属托槽/MBT直丝弓金属托槽5-5（含丝）3带钩 KIT11-322－00CH...,NaN,NaN,NaN,NaN,NaN,NaN,75.24,251.0,上海埃蒙迪,矫正正畸,托槽,金属普通托槽


In [ ]:
# 计算库存变化列和估算金额列
deduplicated_df['库存变化'] = deduplicated_df.apply(lambda row: calculate_stock_change(row['库存_x'], row['库存_y']),axis=1)
    
    
deduplicated_df['估算金额'] = deduplicated_df['库存变化'] * deduplicated_df['折扣价格']